In [3]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [4]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Jamestown,42.0970,-79.2353,50.18,47,44,4.94,US,2022-04-13 01:48:56
1,1,Punta Arenas,-53.1500,-70.9167,44.71,65,40,26.46,CL,2022-04-13 01:48:56
2,2,Hualmay,-11.0964,-77.6139,66.76,86,9,9.69,PE,2022-04-13 01:48:57
3,3,San Pedro,14.3500,121.0167,82.31,83,75,2.30,PH,2022-04-13 01:48:58
4,4,Clyde River,70.4692,-68.5914,-5.57,84,100,4.61,CA,2022-04-13 01:48:58


In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [7]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 80


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,Rikitea,-23.1203,-134.9692,79.27,70,18,18.88,PF,2022-04-13 01:49:02
15,15,Atuona,-9.8000,-139.0333,79.36,79,13,16.87,PF,2022-04-13 01:49:02
19,19,Cap Malheureux,-19.9842,57.6142,77.47,84,19,8.01,MU,2022-04-13 01:49:05
23,23,Dingle,10.9995,122.6711,75.96,85,100,1.03,PH,2022-04-13 01:49:07
26,26,Grand Gaube,-20.0064,57.6608,77.38,84,24,8.01,MU,2022-04-13 01:49:08
31,31,Naliya,23.2667,68.8333,78.73,75,0,12.55,IN,2022-04-13 01:49:10
46,46,Quixeramobim,-5.1992,-39.2928,76.24,82,100,3.42,BR,2022-04-13 01:49:55
56,56,Puerto Ayora,-0.7393,-90.3518,76.89,87,97,5.55,EC,2022-04-13 01:50:14
68,68,Bathsheba,13.2167,-59.5167,79.41,69,40,17.27,BB,2022-04-13 01:50:27
79,79,Singaparna,-7.3515,108.1110,79.83,72,82,3.04,ID,2022-04-13 01:50:32


In [12]:
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       0
Date          0
dtype: int64

In [18]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,San Pedro,PH,82.31,14.3500,121.0167,
5,Champerico,GT,84.25,14.3000,-91.9167,
8,Muncar,ID,83.03,-8.4333,114.3333,
13,Rikitea,PF,79.27,-23.1203,-134.9692,
15,Atuona,PF,79.36,-9.8000,-139.0333,
19,Cap Malheureux,MU,77.47,-19.9842,57.6142,
21,Batticaloa,LK,80.55,7.7102,81.6924,
23,Dingle,PH,75.96,10.9995,122.6711,
25,Vaini,TO,84.36,-21.2000,-175.2000,
26,Grand Gaube,MU,77.38,-20.0064,57.6608,


In [21]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [30]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))